In [1]:
def get_genome_file(genome_location,species):
    list_of_files_in_genome_folder = os.listdir(f"{genome_location}/{species}")
    for file in list_of_files_in_genome_folder:
        if file.endswith("_genomic.fna"):
            genome_file = file
    return(genome_file)  

def get_gene_sequence(genome_location, 
                      species, 
                      genome_file, 
                      scaffold,
                      gene_start,
                      gene_end,
                      complement,
                      output_location):

    
    print("Getting Gene")
    list_of_folders = os.listdir(output_location)
    # print(list_of_files_inside_annotated_species_folder)
    # print(
    if f"0.Temp" not in list_of_folders:
        os.mkdir(f"{output_location}/0.Temp")

#     subprocess.run(f'samtools faidx "{genome_location}/{species}/{genome_file}"', shell = True, stderr = subprocess.DEVNULL)
#     print(f'samtools faidx "{genome_location}/{species}/{genome_file}" {scaffold}:{gene_start}-{gene_end}')
    subprocess.run(f'samtools faidx "{genome_location}/{species}/{genome_file}" {scaffold}:{gene_start}-{gene_end} > "{output_location}/0.Temp/temp_genome.fa"', shell = True, stderr = subprocess.DEVNULL)
    
    genome = SeqIO.parse(f"{output_location}/0.Temp/temp_genome.fa", "fasta")
    for entries in genome:
        gene_sequence = entries.seq
        if complement == "1":
            gene_sequence = gene_sequence.reverse_complement()
        break
    # print(gene_sequence)
    return (gene_sequence)
def make_output_folder(output_location):
    list_of_folders = os.listdir(output_location)
    # if "2.Final_output" in list_of_folders:
    #     subprocess.run(f'rm -r "{output_location}/2.Final_output"', shell = True, stderr = subprocess.DEVNULL)
    # os.mkdir(f"{output_location}/2.Final_output")

def write_exon_output(output_location,species,exon,exon_output):
    
    
    list_of_folders_in_final_output = os.listdir(f"{output_location}/2.Final_output")
    if species not in list_of_folders_in_final_output:
        os.mkdir(f"{output_location}/2.Final_output/{species}")
    
    with open(f"{output_location}/2.Final_output/{species}/{exon}.fa", 'w') as out_file:
        out_file.write(exon_output)

In [4]:
import os
from Bio import SeqIO
import subprocess

family_group_list = ["3.Satyrine","4.Pierinae","5.Coliadinae","6.Heliconiinae_Danainae_Nymphalinae","7.Papilionidae","8.Lycaenidae"]
for family_group in family_group_list:
    output_location = f"/mnt/g/My Drive/Circadian Rhythm Genes Project/11.CWO_Exon_Analysis/{family_group}"
    make_output_folder(output_location)
    genome_location = "/mnt/f/Genomes_2023-12-26"
    list_of_species = os.listdir(f"{output_location}/1.Blast_result")
    if 'desktop.ini' in list_of_species:
        list_of_species.remove("desktop.ini")
    # print(list_of_species)
    # list_of_species = ["Heliconius_charithonia"]
    # list_of_species = ["Maniola_hyperantus"]
    error_exons = ''
    for species in list_of_species:
        print(species)
        genome_file = get_genome_file(genome_location,species)
        print(genome_file)
        previous_coordinate_start = ''
        previous_scaffold = ''
        with open(f"{output_location}/1.Blast_result/{species}/final_coordinates.csv", 'r') as coordinate_final_file:
            coordinate_file_list = coordinate_final_file.readlines()
        
        for i in range(1,len(coordinate_file_list)):
            
            coordinate_line_split = coordinate_file_list[i].split(",")
            
                
    #         print(coodinate_line_split[5])
            if coordinate_line_split[5] == "N":
                if previous_coordinate_start == '':
                    previous_coordinate_start = coordinate_line_split[2]
                    previous_scaffold = coordinate_line_split[1]
                complement = coordinate_line_split[4]
                start,stop = coordinate_line_split[2],coordinate_line_split[3]
                print(start,stop)
                query_name = coordinate_line_split[6]
                scaffold = coordinate_line_split[1]
                exon = f"Exon_{query_name.split('_')[-1]}"
                left_oh, right_oh = query_name.split('_')[-6], query_name.split('_')[-4]
                print(left_oh, right_oh)
    #             print(complement)
                if complement == "0":
                    if start < previous_coordinate_start or scaffold != previous_scaffold:
                        print(f"Error in Coordinates\nPrevious_coordinate = {coordinate_file_list[i-1]}\nCurrent_coordinate = {coordinate_file_list[i]}")
                        prompt = input("Continue (Y/N)?")
                        if prompt[0].lower() == "n":
                            assert False
                elif complement == "1":
                    if start > previous_coordinate_start or scaffold != previous_scaffold:
                        print(f"Error in Coordinates\nPrevious_coordinate = {coordinate_file_list[i-1]}\nCurrent_coordinate = {coordinate_file_list[i]}")
                        prompt = input("Continue (Y/N)?")
                        if prompt[0].lower() == "n":
                            assert False
                else:
                    print("ERROR")
                
            elif coordinate_line_split[5] == "Y":
                query_name = coordinate_line_split[6]
                exon = f"Exon_{query_name.split('_')[-1]}"
                scaffold = coordinate_line_split[1]
                error_exons += f"{species},{scaffold},000,000,0,Y,Error_{exon},00,00,00\n"
                continue
                
            gene_sequence = get_gene_sequence(genome_location, 
                                              species, 
                                              genome_file, 
                                              scaffold,
                                              start,
                                              stop,
                                              complement,
                                              output_location)
            
            gene_sequence_2 = get_gene_sequence(genome_location, 
                                              species, 
                                              genome_file, 
                                              scaffold,
                                              int(start)-2,
                                              int(stop)+2,
                                              complement,
                                              output_location)
        
            current_exon = query_name.split('_')[-1]
            if current_exon != "1":        
                if gene_sequence_2[:2].lower() != "ag":
                    print(species, current_exon)
                    print(gene_sequence)
                    print(gene_sequence_2)
                    print("Not AG")
                    test = input("quit?")
                    if test == "y":
                        assert False
                    # assert False
            print(current_exon, gene_sequence[-3:].translate())
            if gene_sequence[-3:].translate() != "*":
                
                # print(gene_sequence_2[-2:].lower())
                if gene_sequence_2[-2:].lower() != "gt":
                    print(current_exon)
                    print(len(coordinate_file_list))
                    print(species, current_exon)
                    print(gene_sequence)
                    print(gene_sequence_2)
                    print("Not gt")
                    test = input("quit?")
                    if test == "y":
                        assert False
            if current_exon == "1":
                if complement == 1:
                    translated = gene_sequence.reverse_complement.translate()
                else:
                    translated = gene_sequence.translate()
                print(translated)
                if "*" in translated:
                    error_exons += f"{species},{scaffold},000,000,0,Y,Error_{exon},00,00,00\n"
                else:
                    exon_output = f">{species}_{exon}_{scaffold}_{start}_{stop}_left_{left_oh}_right_{right_oh}\n{gene_sequence}"
                    print(exon_output)
            
                    write_exon_output(output_location,species,exon,exon_output)
            elif current_exon == "16":
                if complement == 1:
                    translated = gene_sequence[:-int(left_oh)].reverse_complement.translate()
                    print(gene_sequence)
                else:
                    translated = gene_sequence[int(left_oh):].translate()
                    print(gene_sequence)
    
                if "*" in translated[:-1]:
                    error_exons += f"{species},{scaffold},000,000,0,Y,Error_{exon},00,00,00\n"
                    print(translated)
                else:
                    
                    exon_output = f">{species}_{exon}_{scaffold}_{start}_{stop}_left_{left_oh}_right_{right_oh}\n{gene_sequence}"
                    print(exon_output)
                    write_exon_output(output_location,species,exon,exon_output)
    #         assert False
            else:
                exon_output = f">{species}_{exon}_{scaffold}_{start}_{stop}_left_{left_oh}_right_{right_oh}\n{gene_sequence}"
                print(exon_output)
    
                write_exon_output(output_location,species,exon,exon_output)
            
    #         assert False
            previous_coordinate_start = start
            previous_scaffold = scaffold
        error_exons += "\n\n"
    #     print(coordinate_file_list)
    #     break
    coordinate_file = ''
    print(error_exons)
    with open(f"{output_location}/error_exons.txt", 'w') as error_out_file:
        error_out_file.write(error_exons)

Bicyclus_anynana
GCF_947172395.1_ilBicAnyn1.1_genomic.fna
16174532 16174587
0 2
Getting Gene
Getting Gene
1 Q
MMETRNYWEENVHPVKYD
>Bicyclus_anynana_Exon_1_NC_069089.1_16174532_16174587_left_0_right_2
ATGATGGAAACTCGTAACTATTGGGAGGAAAATGTCCATCCCGTTAAATATGACAA
16140570 16140675
1 0
Getting Gene
Getting Gene
2 K
>Bicyclus_anynana_Exon_2_NC_069089.1_16140570_16140675_left_1_right_0
TTACAACTCCAACGATGGCTTCGGTCGCGGCGAGGCGCACGGCTACGCGCCGCCGTCGGAGGACGAGGCGGAGTACCCGCCCGGCGCGTACCTCAAGAAGGGGAAG
16139077 16139260
0 1
Getting Gene
Getting Gene
3 M
>Bicyclus_anynana_Exon_3_NC_069089.1_16139077_16139260_left_0_right_1
CAAGACCCTATGTCCCACCGGATCATAGAGAAGCGTCGTCGAGACCGCATGAACAACTGCCTGGCCGACCTCTCGCGCCTCATACCACCGGAGTACCTCAAGAAAGGCCGAGGCCGGGTGGAGAAGACAGAGATCATTGAAATGGCAATCAGACATCTCAAGTTCTTGCAGGACCGTGTTAATG
16134878 16135039
2 1
Getting Gene
Getting Gene
4 K
>Bicyclus_anynana_Exon_4_NC_069089.1_16134878_16135039_left_2_right_1
CGGCGGAGCGGCAGGCCTCCGACGAGTTCCGCGGAGGCTACCAGGAGGCCGTGGCTGAGGCCGTCCGCTTCCTCGTCGATGTGCAG

Continue (Y/N)? y


Getting Gene
Getting Gene
5 E
>Phoebis_sennae_Exon_5_LQNK01001443.1_99760_99948_left_2_right_1
GAGAGACGGTACCCCGCGAGAAAACGAGGTCACACCCTGGCTCCTCATCAGAGACGACCAGCTCTTCCAGCAGTTCGCACGGCTTCGCGATGAAAGTGATCCAACAGCCCGAGCCGCCGCCCGCCTTCCCGAGGGAGCCCTACGAACCAGAGACCGTGGAACCGGCGTATCCCATGGAATGTGATAGAG
96197 96390
2 0
Getting Gene
Getting Gene
6 E
>Phoebis_sennae_Exon_6_LQNK01001443.1_96197_96390_left_2_right_0
TAGTAGTCGGGCCACTGCCTCTAGTAAGTAATGAAGCGGCAGCAGAGCCTGAACCCCTCCCCCTCGATGGAAGGCTAAAGCGCGAGCCCACGTTAAGGAAGATTCGCAAACCGGAGCACGGGGAGGATTACCTACATTCCTACAAGTTCAAAAACTCCATAGAAAGGAGGTTTTCGCGATCGCAGGATTCTGAG
94748 94888
0 0
Getting Gene
Getting Gene
7 Q
>Phoebis_sennae_Exon_7_LQNK01001443.1_94748_94888_left_0_right_0
GTGGCGCCACCGGTCGCGCGGCCGGCGCACGCGCACGGCAAGGTGTACTCGCACAAGCGGCGGCGCGCCACCAAGCCCGCGCCCTCCACCTCCACCTCCGCGTCCGGCTCCACGGAGGACGCGCGGGACACGTCGCCGCAG
93506 93596
0 1
Getting Gene
Getting Gene
8 W
>Phoebis_sennae_Exon_8_LQNK01001443.1_93506_93596_left_0_right_1
gaCACATCGAGTGAGTCCCCTCACCATCACCAGTACGACAAGCCG

quit? 


METRHYWEENGHT
>Sericinus_montela_Exon_1_JAGSMR010000053.1_3740440_3740480_left_0_right_2
atggaAACTCGTCATTATTGGGAAGAGAATGGACATACCAT
3761523 3761634
1 0
Getting Gene
Getting Gene
2 K
>Sericinus_montela_Exon_2_JAGSMR010000053.1_3761523_3761634_left_1_right_0
CTTCAACTCGAACGATGGCTTCGGGAGAGTGGGCGGCGAGCAGCTCAGCTTTGCGGCGCCATCGGAGGATGAAGCTGAGTACCCGCCGGGCGCTTATCTGAAGAAGGGGAAG
3763269 3763452
0 1
Getting Gene
Getting Gene
3 M
>Sericinus_montela_Exon_3_JAGSMR010000053.1_3763269_3763452_left_0_right_1
CAAGATCCCATGTCGCACCGCATCATAGAGAAGAGACGCCGTGACCGGATGAACAACTGCTTGGCGGACCTCTCCCGCCTCATCCCGCCCGAGTACCTGAAGAAGGGTCGCGGCAGAGTGGAGAAAACAGAGATAATAGAAATGGCGATACGCCATTTGAAGTTTTTGCAGGAGCGCGCCAATG
3764759 3764920
2 1
Getting Gene
Getting Gene
4 K
>Sericinus_montela_Exon_4_JAGSMR010000053.1_3764759_3764920_left_2_right_1
CTAATGAAAGAGAGGCGGGCGAAGTGCATTTCCGTAATGGTTACCAAGAGGCGGTCGGCGAGGCGATACGCTACCTGGCCGAGGCGCAGGGCTTCCCCGCGGATGGGCTCTGCGCGCAGCTCGCCAATCATCTCCAGAGGCATTGCGACATCATCACAAAAG
3765600 3765785
2 1
Getting Gene


...

Getting Gene
Getting Gene
2 K
>Calephelis_nemesis_Exon_2_NJDD01001685.1_65369_65480_left_1_right_0
TTACAATGCCAATGATGACTACGGCCGCGTTCCCGGAGAGCCGCTACACTTCGCGCCACCTTCAGAAGATGAGGCCGAGTACCCGCCGGGTGCTTATCTCAAGAAGGGGAAG
63890 64073
0 1
Getting Gene
Getting Gene
3 M
>Calephelis_nemesis_Exon_3_NJDD01001685.1_63890_64073_left_0_right_1
CAAGATCCGATGTCGCACAGAATTATAGAAAAACGACGTCGCGACAGAATGAACAATTGTCTAGCAGATCTGTCTAGACTTATACCACCTGAATATTTAAAGAAGGGTCGTGGAAGAGTCGAGAAGACAGAGATAATTGAAATGGCAATACGCCATCTTAAGTATTTACAAGATAGAGTGAATG
59665 59832
2 1
Getting Gene
Getting Gene
4 K
>Calephelis_nemesis_Exon_4_NJDD01001685.1_59665_59832_left_2_right_1
ATGCCGAGAGACAAGCTGCAGAAGATCATTTCAGGGCTGGCTACCAAGAAGCGGTGACTGAAGTTGTGCGATTCCTAATAGAAGTTCAAGGGTTTGCGCCTGGCGATGGACTTTGTGTTCAACTTGGCTCTCATCTACAAAGGCACTGTGAAAGTGTTACTTCTaaag
58457 58645
2 1
Getting Gene
Getting Gene
5 E
>Calephelis_nemesis_Exon_5_NJDD01001685.1_58457_58645_left_2_right_1
GTTCAACGTTACCTCGTGAAAAACGATCTCATATTGGCTCATCATCAGAAACAGCAAGTTCATCAAGTAGCTCCCACAGTTACGCTGTC

Continue (Y/N)? y


Getting Gene
Getting Gene
2 K
>Calephelis_virginiensis_Exon_2_NJDC01000311.1_101525_101636_left_1_right_0
ttaCAATACCAATGACGACTATGGCCGCGTTCCCGGAGAGCCCCTATACTTCGCGCCACCTTCAGAAGATGAGGCCGAGTACCCGCCGGGTGCTTATCTCAAAAAGGGGAAG
102944 103127
0 1
Getting Gene
Getting Gene
3 M
>Calephelis_virginiensis_Exon_3_NJDC01000311.1_102944_103127_left_0_right_1
CAAGATCCGATGTCGCACAGAATTATAGAAAAACGACGTCGCGACAGAATGAACAATTGTCTAGCAGATCTGTCTAGACTTATACCACCTGAATATTTAAAGAAGGGCCGTGGAAGAGTCGAGAAGACAGAGATAATTGAAATGGCAATACGCcatcttaaatatttacaagataGAGTAAATG
107630 107797
2 1
Getting Gene
Getting Gene
4 K
>Calephelis_virginiensis_Exon_4_NJDC01000311.1_107630_107797_left_2_right_1
ATGCCGAGAGACAGGCTGCAGAAGATCATTTCAGGGCTGGCTACCAAGAAGCGGTGACGGAAGTCGTAAGATTCCTAATAGAAGTTCAAGGATTTGCGCCGGGCGATGGACTCTGTGTTCAACTTGGCTCTCATCTGCAAAGGCACTGTGAAAGTGTTACTAGTAAAG
108719 108907
2 1
Getting Gene
Getting Gene
5 E
>Calephelis_virginiensis_Exon_5_NJDC01000311.1_108719_108907_left_2_right_1
GTTCAACGTTACCTCGTGAAAAACGATCTCATATGGGCTCATCATCAGAAACCGC

quit? 


>Lycaena_phlaeas_Exon_6_HG995173.1_2493146_2493336_left_2_right_0
TGGCGGTACAGCAGTTGATAATTCCCGTGGAGAGTTTGCCCTCCGACGAGCCCCTGCCCCTGGACGGGAGACACAAGCGCGAGCCCACCCTGAGGAAGGTGCGCAAGCCGGACCACGCTGACGACTACCTACATTCCTATAAGTTTAAGAACTCCATCGAACGCCGCTTCTCGAGGTCGCAAGATTCAGAG
2493623 2493751
0 0
Getting Gene
Getting Gene
7 Q
>Lycaena_phlaeas_Exon_7_HG995173.1_2493623_2493751_left_0_right_0
gtaGCAGAAAACTCGACGCGAAACGCTCGAGCGTTTACAAAGCGAAGGCGTGCGACAAAACCCGTGCCCGCCTCCGCTTCCGCCTCTACTTCGGGCTCCACTGAAGACGCGCGAGATACCTCGCCGCAG
2494590 2494677
0 1
Getting Gene
Getting Gene
8 S
>Lycaena_phlaeas_Exon_8_HG995173.1_2494590_2494677_left_0_right_1
GACACGTCAAGCGAGTCGACTCACCATCACCAGTACGAGAAGCCGATGCCGCAACAGTACGTGCCCGTGTTCGCGTTGAACGCCCTCG
2494977 2495182
2 0
Getting Gene
Getting Gene
9 *
>Lycaena_phlaeas_Exon_9_HG995173.1_2494977_2495182_left_2_right_0
GCAAGTACTACGTGCCGCTGAGCGTGGAGTACGCGTGCCTGGCGCGGCTGGGCGCGGGCGCGTTCGACGCGCTGGCCGCGCGCGCCGTGCCGCCCGCCGCGCCGCTGCACCCCGTCACCATACACGTCAATTTCCAGCCCTGCGCCGACTACCACGTCAAGCGCGAACCCGACTG